In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Paths
path_md = './Recipes-MD/'
output_folder = './Standardized_Recipes/'

# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  # Example: "https://<your-resource-name>.openai.azure.com/"
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_name = "gpt-4"  # Use the model you need

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key=azure_api_key,
    api_version="2024-07-01-preview",  # Set the correct version
    azure_endpoint=azure_endpoint
)

# Template JSON structure (same as before)
template_json = {
    "id": "recipe-id",
    "name": "Recipe Name",
    "prep_time": "Prep Time",
    "cooking_time": "Cooking Time",
    "difficulty": "Difficulty Level",
    "serving_size": "Serving Size",
    "calories_per_serving": "Calories per Serving",
    "dietary_info": "Dietary Information",
    "ingredients": [
        {
            "name": "Ingredient Name",
            "quantity": "Quantity",
            "allergens": ["Allergen 1", "Allergen 2", "Allergen 3"],
            "image_url": "https://example.com/images/recipe-name-ingredient-name.png"
        }
    ],
    "not_included_in_delivery": [
        {
            "name": "Ingredient Name",
            "quantity": "Quantity",
            "image_url": "https://example.com/images/recipe-name-ingredient-name.png"
        }
    ],
    "cooking_steps": [
        {
            "step": 1,
            "description": "Step description",
            "image_url": "https://example.com/images/recipe-name-step-1.png"
        }
    ],
    "nutrition_values": {
        "per_serving": {
            "calories": "Calories",
            "fat": "Fat Amount",
            "saturated_fat": "Saturated Fat",
            "carbohydrate": "Carbohydrate",
            "sugar": "Sugar Amount",
            "dietary_fiber": "Fiber Amount",
            "protein": "Protein Amount",
            "cholesterol": "Cholesterol Amount",
            "sodium": "Sodium Amount",
            "potassium": "Potassium Amount",
            "calcium": "Calcium Amount",
            "iron": "Iron Amount"
        }
    },
    "tags": [
        "Tag 1",
        "Tag 2",
        "Tag 3"
    ],
    "image_url": "https://example.com/images/recipe-name-main.png"
}

def generate_text(prompt):
    try:
        # Call Azure OpenAI API
        response = client.completions.create(
            model=model_name,
            prompt=prompt,
            max_tokens=2000,  # Adjust based on the length of your recipes
            temperature=0.7,  # Adjust based on your need for creativity
            n=1
        )
        return response['choices'][0]['text'].strip()
    except Exception as e:
        return f"Error: {e}"

# Process each Markdown file
for filename in os.listdir(path_md):
    if filename.endswith(".md"):
        base_name = os.path.splitext(filename)[0]
        file_path = os.path.join(path_md, filename)
        
        print(f"Processing file: {filename}")
        
        try:
            with open(file_path, "r", encoding="utf-8") as md_file:
                md_content = md_file.read()
        except Exception as e:
            print(f"Failed to load Markdown file: {filename}, Error: {e}")
            continue

        # Prepare input for Azure OpenAI
        prompt = (
            "You are a helpful assistant that standardizes recipe data. "
            "Return the Input Recipe in the Template JSON format: \n\n"
            f"Template: {json.dumps(template_json, indent=4)}\n\n"
            f"Input Recipe:\n{md_content}"
        )

        print(f"Prompt for Azure OpenAI: {prompt}")

        # Call Azure OpenAI API
        organized_content = generate_text(prompt)
        print(f"Raw response: {organized_content}")

        # Validate and parse the JSON response
        try:
            standardized_recipe = json.loads(
                organized_content.replace("```json\n", "").replace("```", "")
            )
        except json.JSONDecodeError:
            print(f"Failed to parse JSON response for file: {filename}")
            continue

        # Save the standardized recipe to a new JSON file
        output_file_path = os.path.join(output_folder, f"standardized_{base_name}.json")
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            json.dump(standardized_recipe, output_file, indent=4)

        print(f"Standardized recipe saved to: {output_file_path}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: './Recipes-MD/'

In [1]:
import os
from openai import AzureOpenAI